In [ ]:
!pip install autoawq

In [ ]:
from huggingface_hub import notebook_login as hfl
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datetime import datetime
from pynvml import *
import awq
import torch

In [ ]:
hfl()

In [ ]:
def printGPUState():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    
    print(f"GPU Memory Occupied: {info.used // 1024 ** 2} MB.")

In [ ]:
%%time
# model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
# model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP"
model_id = "0xVolt/Code-Mistral-Llama-7B-TIES"

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    device_map={"": 0}
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

In [ ]:
printGPUState()

In [ ]:
def generateOutputWithModelPipeline(model, tokenizer, prompt, temperature=0.7): 
    pipe = pipeline(
        task="text-generation", 
        model=model, 
        tokenizer=tokenizer
    )
    
    start = datetime.now()
    
    output = pipe(
        prompt,
        do_sample=True,
#         max_new_tokens=3000, 
        temperature=temperature, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1
    )
    
    stop = datetime.now()
    
    totalTimeToPrompt = stop - start
    print(f"Execution Time : {totalTimeToPrompt}")
    
    return output

In [ ]:
prompt = """
Write a function in C++ that appends an element to the end of a linked list. Explain your code with comments as necessary.
"""

In [ ]:
%%time
output = generateOutputWithModelPipeline(model, tokenizer, prompt)
print(output["generated_text"])

In [ ]:
output